In [52]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm 

#load course IDs
course_ids = pd.read_csv('data/course_IDs.csv')
course_ids = course_ids['course_ID'].tolist()
course_ids = [str(i) for i in course_ids]


In [53]:
TESTING = True

import random
random.seed(123)
course_ids_test = random.sample(course_ids, 20)


In [56]:
def get_details(soup, url):
    # get the course name
    title_article = soup.find('article', class_='content')
    year = soup.find('div', class_='edition').text.strip()
    
    course_title = soup.find('h1').text.strip()

    try:
        language = soup.find('dt', string='Language')
        language = language.find_next_sibling().text.strip()
    except:
        language = 'None'
    
    try:
        college = soup.find('dt', string='Faculty')
        college = college.find_next_sibling().text.strip()
    except:
        college = 'None'

    try:
        lecturers = soup.find('dt', string={'Teacher', 'Teachers'})
        lecturers = lecturers.find_next_sibling().text.strip()
        lecturers = lecturers.split('\n')
    except:
        lecturers = 'None'

    try:
        lecturers_urls = soup.find('dt', string={'Teacher', 'Teachers'})
        lecturers_urls_container = lecturers_urls.find_next_sibling()
        lecturers_urls = lecturers_urls_container.find_all('a')
        #get links
        if len(lecturers_urls) == 0:
            lecturers_urls = 'None'
        else:
            lecturers_urls = [i['href'] for i in lecturers_urls]
    except:
        lecturers_urls = 'None'

    try:
        programmes_prev = soup.find('h2', string='Part of')
        programmes_container = programmes_prev.find_next_sibling('ul')
        programmes = programmes_container.find_all('a')
        #get links
        programmes = [i['href'] for i in programmes]
    except:
        programmes = 'None'

    try:
        text_container = soup.find('div', class_='wrapper main clearfix')
        text = text_container.find('article', class_='content').text.strip()
    except:
        text = 'None'

    #create dictionary from course details
    course_details = {
        'url': url,
        'year': year,
        'course_title': course_title,
        'language': language,
        'college': college,
        'lecturers_urls': lecturers_urls,
        'lecturers': lecturers,
        'programmes': programmes,
        'text': text
    }

    return course_details

In [57]:
url = 'https://studiegids.universiteitleiden.nl/en/courses/'

# create empty df
course_details_df = pd.DataFrame(columns=['url', 'year','course_title','language','college','lecturers_urls','lecturers','programmes','text'])

# loop over all course IDs
for i in tqdm(course_ids_test if TESTING else course_ids):
    # get the html code of the course page
    url_new = url + i
    response = requests.get(url_new)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    course_details = get_details(soup, url_new)
    
    #append dictionary to df
    course_details_df = course_details_df._append(course_details, ignore_index=True)

#save df to csv
course_details_df.to_csv('data/course_details.csv', index=False)

  0%|          | 0/20 [00:00<?, ?it/s]